In [1]:
!pip install rioxarray

     |████████████████████████████████| 47 kB 1.4 MB/s             
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54590 sha256=0600c623431f376229ceac06a3ee1b75098412b60ae582391fc963b6265c9f39
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarray


In [2]:
import os
import pandas as pd
import numpy as np
import rasterio as rio
import rioxarray as rxr
import warnings
warnings.filterwarnings('ignore', 'RuntimeWarning', )

In [3]:
havza_ndvil89 = os.listdir("../input/2204-d-havza-ndvi-l89")
il_ndvil89 = os.listdir("../input/2204-d-l-s-n-rlar-na-k-rpma-l89")
ilce_ndvil89 = os.listdir("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l89")

havza_ndvil5 = os.listdir("../input/2204-d-havza-ndvi-l5")
il_ndvil5 = os.listdir("../input/2204-d-l-s-n-rlar-na-k-rpma-l5")
ilce_ndvil5 = os.listdir("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l5")

havza_ndvil89.sort()
il_ndvil89.sort()
ilce_ndvil89.sort()

havza_ndvil5.sort()
il_ndvil5.sort()
ilce_ndvil5.sort()

havza_ndvil89

['Havza_2013_ndvi.tif',
 'Havza_2014_ndvi.tif',
 'Havza_2015_ndvi.tif',
 'Havza_2016_ndvi.tif',
 'Havza_2017_ndvi.tif',
 'Havza_2018_ndvi.tif',
 'Havza_2019_ndvi.tif',
 'Havza_2020_ndvi.tif',
 'Havza_2021_ndvi.tif',
 '__notebook__.ipynb',
 '__output__.json',
 '__results__.html',
 '__results___files',
 'custom.css']

In [4]:
il_ndvil5

['Year2000',
 'Year2001',
 'Year2002',
 'Year2003',
 'Year2004',
 'Year2005',
 'Year2006',
 'Year2007',
 'Year2008',
 'Year2009',
 'Year2010',
 'Year2011',
 '__notebook__.ipynb',
 '__output__.json',
 '__results__.html',
 '__results___files',
 'custom.css']

In [5]:
havza_ndvil89 = havza_ndvil89[:-5]
il_ndvil89 = il_ndvil89[:-5]
ilce_ndvil89 = ilce_ndvil89[:-5]
havza_ndvil5 = havza_ndvil5[:-5]
il_ndvil5 = il_ndvil5[:-5]
ilce_ndvil5 = ilce_ndvil5[:-5]
havza_ndvil89

['Havza_2013_ndvi.tif',
 'Havza_2014_ndvi.tif',
 'Havza_2015_ndvi.tif',
 'Havza_2016_ndvi.tif',
 'Havza_2017_ndvi.tif',
 'Havza_2018_ndvi.tif',
 'Havza_2019_ndvi.tif',
 'Havza_2020_ndvi.tif',
 'Havza_2021_ndvi.tif']

In [6]:
il_ndvil5

['Year2000',
 'Year2001',
 'Year2002',
 'Year2003',
 'Year2004',
 'Year2005',
 'Year2006',
 'Year2007',
 'Year2008',
 'Year2009',
 'Year2010',
 'Year2011']

In [7]:
ndvi_class_bins = [0, 0.2, 0.5, 1.0]
def ndvi_to_csvdict(ndvi_arr, year, place):
     # NDVI ve ilgili değerlerin hesaplanması için
    cat1 = 0
    cat2 = 0
    cat3 = 0
    
    n_mean = np.nanmean(ndvi_arr) # Ortalama
    n_std = np.nanstd(ndvi_arr) # Standart Sapma
    
    # Kategorikleştirme için
    ndvi_landsat_class = np.digitize(ndvi_arr, ndvi_class_bins)
    ndvi_landsat_class = np.ma.masked_where(np.ma.getmask(ndvi_arr), ndvi_landsat_class)
    np.unique(ndvi_landsat_class)
    comp_class = ndvi_landsat_class.compressed()
    
    for piksel in comp_class:
        if piksel == 1:
            cat1+=1
        elif piksel ==2:
            cat2+=1
        elif piksel == 3:
            cat3+=1
    top = cat1+ cat2+ cat3
    # Yüzdeleri için
    try:
        cat1y = cat1 / top * 100
        cat2y = cat2 / top * 100
        cat3y = cat3 / top * 100
        
        result = {"yer" : place, "yıl": year, "ort" : n_mean, "std" : n_std,
                  "kat1" : cat1y, "kat2" : cat2y, "kat3" : cat3y}
    
        return result
    
    except ZeroDivisionError:
        print("Boş Görüntü Atlandı")
        return "None"


    """
    kat1 = 0.0 < NDVI < 0.2 olan piksellerin bulunma yüzdesi
    kat2 = 0.2 < NDVI < 0.5 olan piksellerin bulunma yüzdesi
    kat3 = 0.5 < NDVI olan piksellerin bulunma yüzdesi
    """

In [8]:
dictlist = []
for ndvi in havza_ndvil89:
    year = ndvi.split("_")[1]
    ndvi_arr = rxr.open_rasterio("../input/2204-d-havza-ndvi-l89/"+ndvi, masked=True).squeeze()
    dictlist.append(ndvi_to_csvdict(ndvi_arr, year = year, place = "Havza"))

In [9]:
dictlist[0]

{'yer': 'Havza',
 'yıl': '2013',
 'ort': 0.14240573,
 'std': 0.089891754,
 'kat1': 83.46097849664432,
 'kat2': 15.849318002251122,
 'kat3': 0.6897035011045611}

In [10]:
for ndvi in havza_ndvil5:
    year = ndvi.split("_")[1]
    ndvi_arr = rxr.open_rasterio("../input/2204-d-havza-ndvi-l5/"+ndvi, masked=True).squeeze()
    dictlist.append(ndvi_to_csvdict(ndvi_arr, year = year, place = "Havza"))

In [11]:
dictlist[-1]

{'yer': 'Havza',
 'yıl': '2011',
 'ort': 0.0977312,
 'std': 0.06280664,
 'kat1': 93.48986777998054,
 'kat2': 6.50762590731479,
 'kat3': 0.0025063127046700702}

In [12]:
for yearlist in il_ndvil89:
    year = yearlist[-4:]
    for ndvi in os.listdir("../input/2204-d-l-s-n-rlar-na-k-rpma-l89/"+ yearlist):
        place = ndvi.split("_")[0:2]
        place = place[0]+"_"+place[1]
        print(place)

Ankara_il
Antalya_il
Karaman_il
Nevsehir_il
Nigde_il
Aksaray_il
Isparta_il
Konya_il
Mersin_il
Mersin_il
Antalya_il
Nevsehir_il
Nigde_il
Ankara_il
Karaman_il
Konya_il
Isparta_il
Aksaray_il
Karaman_il
Nigde_il
Mersin_il
Antalya_il
Isparta_il
Aksaray_il
Nevsehir_il
Konya_il
Ankara_il
Aksaray_il
Nevsehir_il
Isparta_il
Karaman_il
Mersin_il
Antalya_il
Nigde_il
Ankara_il
Konya_il
Ankara_il
Mersin_il
Antalya_il
Nigde_il
Nevsehir_il
Aksaray_il
Konya_il
Karaman_il
Isparta_il
Nigde_il
Antalya_il
Nevsehir_il
Mersin_il
Isparta_il
Aksaray_il
Konya_il
Karaman_il
Ankara_il
Nevsehir_il
Nigde_il
Aksaray_il
Mersin_il
Ankara_il
Karaman_il
Antalya_il
Isparta_il
Konya_il
Karaman_il
Antalya_il
Nigde_il
Aksaray_il
Isparta_il
Mersin_il
Ankara_il
Konya_il
Nevsehir_il
Nigde_il
Isparta_il
Aksaray_il
Mersin_il
Nevsehir_il
Karaman_il
Antalya_il
Konya_il
Ankara_il


In [13]:
for yearlist in il_ndvil89:
    year = yearlist[-4:]
    for ndvi in os.listdir("../input/2204-d-l-s-n-rlar-na-k-rpma-l89/"+ yearlist):
        place = ndvi.split("_")[0:2]
        place = place[0]+"_"+place[1]
        ndvi_arr = rxr.open_rasterio("../input/2204-d-l-s-n-rlar-na-k-rpma-l89/"+yearlist+"/"+ndvi, masked=True).squeeze()
        dictlist.append(ndvi_to_csvdict(ndvi_arr, year = year, place = place))

In [14]:
dictlist[-1]

{'yer': 'Ankara_il',
 'yıl': '2021',
 'ort': 0.11313289,
 'std': 0.056415703,
 'kat1': 93.74673725080753,
 'kat2': 6.156144980260367,
 'kat3': 0.09711776893210218}

In [15]:
for yearlist in il_ndvil5:
    year = yearlist[-4:]
    for ndvi in os.listdir("../input/2204-d-l-s-n-rlar-na-k-rpma-l5/"+ yearlist):
        place = ndvi.split("_")[0:2]
        place = place[0]+"_"+place[1]
        ndvi_arr = rxr.open_rasterio("../input/2204-d-l-s-n-rlar-na-k-rpma-l5/"+yearlist+"/"+ndvi, masked=True).squeeze()
        dictlist.append(ndvi_to_csvdict(ndvi_arr, year = year, place = place))

In [16]:
dictlist[-1]

{'yer': 'Nevsehir_il',
 'yıl': '2011',
 'ort': 0.11956179,
 'std': 0.059083033,
 'kat1': 90.56813723845956,
 'kat2': 9.430696065463316,
 'kat3': 0.0011666960771302777}

In [17]:
for yearlist in ilce_ndvil89:
    year = yearlist[-4:]
    for ndvi in os.listdir("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l89/"+ yearlist):
        place = ndvi.split("_")[0]
        ndvi_arr = rxr.open_rasterio("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l89/"+yearlist+"/"+ndvi, masked=True).squeeze()
        dictlist.append(ndvi_to_csvdict(ndvi_arr, year = year, place = place))

In [18]:
print("ilçeee")

ilçeee


In [19]:
dictlist[-1]

{'yer': 'Seydisehir',
 'yıl': '2021',
 'ort': 0.2151978,
 'std': 0.07011963,
 'kat1': 46.70212005586039,
 'kat2': 53.2886019507153,
 'kat3': 0.009277993424308068}

In [20]:
for yearlist in ilce_ndvil5:
    year = yearlist[-4:]
    for ndvi in os.listdir("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l5/"+ yearlist):
        place = ndvi.split("_")[0]
        ndvi_arr = rxr.open_rasterio("../input/2204-d-l-e-s-n-rlar-na-k-rpma-l5/"+yearlist+"/"+ndvi, masked=True).squeeze()
        csvdict = ndvi_to_csvdict(ndvi_arr, year = year, place = place)
        if csvdict == "None":
            continue
        else:
            dictlist.append(csvdict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Boş Görüntü Atlandı


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Boş Görüntü Atlandı


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


...

Boş Görüntü Atlandı


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Boş Görüntü Atlandı


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Boş Görüntü Atlandı


In [21]:
dictlist[-1]

{'yer': 'Meram',
 'yıl': '2011',
 'ort': 0.11431679,
 'std': 0.06969182,
 'kat1': 90.3282746123152,
 'kat2': 9.665688472751341,
 'kat3': 0.006036914933457704}

In [22]:
dictlist[100]

{'yer': 'Konya_il',
 'yıl': '2021',
 'ort': 0.15037057,
 'std': 0.0900481,
 'kat1': 80.27865744219098,
 'kat2': 19.15757793490089,
 'kat3': 0.5637646229081307}

In [23]:
print("dataframe e gidiyok")

dataframe e gidiyok


In [24]:
ndvi_df = pd.DataFrame(dictlist)
ndvi_df

,yer,yıl,ort,std,kat1,kat2,kat3
0,Havza,2013,0.142406,0.089892,83.460978,15.849318,0.689704
1,Havza,2014,0.137440,0.087038,82.974731,16.656456,0.368813
2,Havza,2015,0.145719,0.087256,82.065228,17.313620,0.621152
3,Havza,2016,0.143232,0.103975,83.036858,15.607313,1.355829
4,Havza,2017,0.171333,0.098933,71.907482,27.255065,0.837452
...,...,...,...,...,...,...,...
1579,Mut,2011,0.093986,0.015203,99.997313,0.002687,0.000000
1580,Toroslar,2011,0.096852,0.023959,99.960830,0.039170,0.000000
1581,Sultanhani,2011,0.079294,0.056371,96.059978,3.926262,0.013760
1582,Guzelyurt,2011,0.114842,0.055777,92.318737,7.680925,0.000338


In [25]:
ndvi_df.to_csv("./NDVI_Degerler_2000_2021.csv")